<a href="https://colab.research.google.com/github/Ksenia-90/recommender_systems/blob/lesson_33/HW_33.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Урок 3. Коллаборативная фильтрация

Ответы на вопросы:

Мне кажется необходимо сегментировать клиентов на более мелкие группы, разделяя по признакам: пол, возраст, география, поведение на сайте, интересующие товары, история покупол, кластиризация товара. Важна чатота рассылок и рекоментации конкретному клиенту. Можно предположить что продажи выратут на 15-20%. Можно пойти от обратного выбрать нужный товар для продажи и подобрать группу клиентов которая с большей вероятностью купить этот товар. Например: Kомпания продает товары для животных через интернет-магазин. Алгоритм анализирует частотность конкретных покупок конкретными клиентами. Предположим, что корм покупают один раз в месяц. Учитывая персональные особенности и индивидуальный график каждого клиента, отправляем ему письмо с предложением повторить предыдущий заказ. Получаем например результат: +10% click rate +32% конверсии в заказ

Очень важно уделить внимание бизнес аспекту например: не следует часто менять цены, чтобы не смутить клиентов; не стоит в несколько раз изменять стоимость товара, Индивидуальное ценообразование для группы клиентов в одном сегменте покупок, географии, обьеме покупок.....

Основные методы: Контент-ориентированный подход. Коллаборативная фильтрация. Кластеризация.

Важно погрузиться в суть бизнеса не всегда применяются одни и теже методы все индивидуально: где-то лучше будут работать рекомендации по подобию, где-то по сопутствующим товарам, где-то по связи покупок клиентов.

Поэкспериментировать с ALS

In [ ]:
model = AlternatingLeastSquares(factors=100, 
                                regularization=0.01,
                                use_native=True,
                                use_cg=True,
                                iterations=15,
                                use_gpu=implicit.gpu.HAS_CUDA,
                                calculate_training_loss=False,
                                num_threads=2
                                )

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

Написать функцию

In [ ]:
  def prefilter_items(data, take_n_popular=5000):
    """Предфильтрация товаров"""
    data = data[data.sales_value / data.quantity >= 1]

    data = data[data.sales_value != 0]
    data = data[data.quantity != 0]
    data = data[data.sales_value / data.quantity <= 30]

    popular = data.groupby('item_id')['user_id'].nunique().reset_index()
    users_count = data['user_id'].nunique()
    popular['user_id'] = popular['user_id'].apply(lambda x: x / users_count)
    popular.rename(columns={'user_id': 'share_unique_users'}, inplace=True)
    popular.sort_values(by='share_unique_users', ascending=False, inplace=True)

    top_popular = popular[popular['share_unique_users'] > 0.5].item_id.tolist()
    data = data[~data['item_id'].isin(top_popular)]
    
    popularity = data.groupby('item_id')['quantity'].sum().reset_index()
    popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

    top_n = popularity.sort_values('n_sold', ascending=False).head(take_n_popular).item_id.tolist()
   
   
    data.loc[~data['item_id'].isin(top_n), 'item_id'] = 999999

    return data